# Projeto de Treinamento de uma LLM com dados próprios

Para treinar uma LLM com dados próprios, preciso fazer um fine tune. Os dados para esse fine-tune serão obtidos através de perguntas e respostas. Para isso, preciso treinar uma llm para gerar perguntas para um determinado contexto.



1. <s>Obter uma base de dados com contextos gerais em portugues</s>
2. <s>Utilizar os contextos e pedir ao gpt4 para gerar perguntas para esses contextos</s> usar `nunorc/squad_v1_pt`
3. Utilizar os pares contextos perguntas e respostas para treinar uma LLM para gerar perguntas dados os contextos 
4. Utilizar a LLM treinada para gerar perguntas para os contextos dos documentos internos
5. Utilizar os pares de perguntas e respostas para treinar a nova llm

nunorc/squad_v1_pt

In [8]:
!pip uninstall transformers -y -qqq
!pip install transformers==4.44.0 -qqq

In [49]:
!pip install flash-attn --no-build-isolation -qqq

In [1]:
import logging
import warnings

# Suprimir avisos específicos de FutureWarning e UserWarning
warnings.filterwarnings("ignore", category=FutureWarning, message=".*TRANSFORMERS_CACHE.*")
warnings.filterwarnings("ignore", message=".*resume_download.*deprecated.*", category=FutureWarning)
warnings.filterwarnings("ignore", message=".*use_cache=True.*", category=UserWarning)
warnings.filterwarnings("ignore", message=".*use_reentrant parameter should be passed explicitly.*", category=UserWarning)

# Configurar o nível de log para a biblioteca transformers
logging.getLogger("transformers.trainer").setLevel(logging.WARNING)
logging.getLogger("transformers.trainer_utils").setLevel(logging.WARNING)
logging.getLogger("transformers.training_args").setLevel(logging.WARNING)


In [2]:
import torch
import bitsandbytes
import peft
import accelerate
import datasets
import trl
import warnings
import transformers

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("torch version:", torch.__version__)
print("bitsandbytes version:", bitsandbytes.__version__)
print("peft version:", peft.__version__)
print("accelerate version:", accelerate.__version__)
print("datasets version:", datasets.__version__)
print("trl version:", trl.__version__)
print("transformers version:", transformers.__version__)
print(f"Device name: '{torch.cuda.get_device_name()}'")
print("Device:", device)
print(f"Device properties: '{torch.cuda.get_device_properties(torch.cuda.current_device())}'")
print("Suporta bfloat16." if torch.cuda.is_bf16_supported() else "Não suporta bfloat16.")

torch version: 2.3.1
bitsandbytes version: 0.43.1
peft version: 0.11.1
accelerate version: 0.31.0
datasets version: 2.19.2
trl version: 0.9.4
transformers version: 4.44.0
Device name: 'NVIDIA GeForce RTX 2060 SUPER'
Device: cuda
Device properties: '_CudaDeviceProperties(name='NVIDIA GeForce RTX 2060 SUPER', major=7, minor=5, total_memory=7957MB, multi_processor_count=34)'
Suporta bfloat16.


In [3]:
import os
from random import randrange

import torch
import numpy as np
import pandas as pd
from huggingface_hub import login
from datasets import load_dataset, Dataset


from trl import SFTConfig, SFTTrainer
from peft import LoraConfig, prepare_model_for_kbit_training, TaskType, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    TrainerCallback,
    set_seed,
    pipeline,
    TrainerCallback,
    TrainerControl,
    TrainerState,
)

# Funcoes

In [4]:
class SaveCheckpointCallback(TrainerCallback):
    def on_save(self, args, state, control, **kwargs):
        print(f"Saving checkpoint at step {state.global_step}")

class EarlyStoppingCallback(TrainerCallback):
    def __init__(self, early_stopping_patience=3, early_stopping_threshold=0.02):
        self.early_stopping_patience = early_stopping_patience
        self.early_stopping_threshold = early_stopping_threshold
        self.best_metric = None
        self.counter = 0

    def on_evaluate(self, args, state: TrainerState, control: TrainerControl, metrics=None, **kwargs):
        current_metric = metrics.get("eval_loss")  # Use the relevant metric for your task

        if current_metric is None:
            return

        if self.best_metric is None or current_metric < self.best_metric - self.early_stopping_threshold:
            self.best_metric = current_metric
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.early_stopping_patience:
                control.should_training_stop = True
                print(f"Early stopping at step {state.global_step} with best eval_loss = {self.best_metric}")


class SaveMetricsCallback(TrainerCallback):
    def __init__(self, output_dir):
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        self.output_dir = output_dir
        self.output_path = os.path.join(output_dir, "metrics.json")
        self.state_path = os.path.join(output_dir, "state.json")
        print(f"Output directory initialized at {output_dir}")
        self.metrics = self.load_existing_metrics()

    def load_existing_metrics(self):
        if os.path.isfile(self.output_path):
            return pd.read_json(self.output_path, lines=True).to_dict('records')
        return []

    def on_step_end(self, args, state: TrainerState, control: TrainerControl, **kwargs):
        state.save_to_json(self.state_path)
    
    def on_evaluate(self, args, state: TrainerState, control: TrainerControl, metrics=None, **kwargs):
        if metrics:
            step = state.global_step
            _metrics = {"Step": step, **metrics}
            self.metrics.append(_metrics)
            metrics_df = pd.DataFrame(self.metrics).drop_duplicates(subset=['Step'], keep='last')
            metrics_df.to_json(self.output_path, orient="records", lines=True)



In [5]:
def get_latest_checkpoint(checkpoint_dir):
    try:
        checkpoints = [os.path.join(checkpoint_dir, d) for d in os.listdir(checkpoint_dir) if d.startswith("checkpoint-")]
        if not checkpoints:
            return None
        latest_checkpoint = max(checkpoints, key=os.path.getctime)
        return latest_checkpoint

    except FileNotFoundError:
        return None

In [6]:
def set_tokenizer(model_id):
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.padding_side = 'right'
    return tokenizer

# Importar os dados do SQUAD em portugues

In [7]:
# df = load_dataset("hltcoe/megawika", "pt") # Demora muito

In [7]:
dataset = load_dataset("nunorc/squad_v1_pt")
dataset = dataset.map(lambda row: {"response": row['answers']['text'][0] })


columns = ['id', 'context', 'question','response']
train_dataset = dataset["train"].shuffle(42).select(range(1000)).select_columns(columns)
test_dataset = dataset["validation"].shuffle(42).select(range(1000)).select_columns(columns)

# Preparar modelo

In [8]:
if torch.cuda.is_bf16_supported():
    compute_dtype = torch.bfloat16
    # attn_implementation = 'flash_attention_2'
    attn_implementation = 'eager'
else:
    compute_dtype = torch.float16
    attn_implementation = 'eager'

print(attn_implementation)
print(compute_dtype)

eager
torch.bfloat16


In [9]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# os.environ['TOKENIZERS_PARALLELISM'] = 'true'
# os.environ['TORCH_USE_CUDA_DSA'] = "1"

model_id = "Qwen/Qwen2-0.5B"
LOCAL_MODELPATH = "models/question-generator/" + model_id.lower().replace("/","-").replace(".","_")
login(token=os.environ.get("HUGGINGFACE_TOKEN"))
set_seed(1234)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [10]:
# A quantização é uma técnica para reduzir o tamanho do modelo e aumentar a eficiência computacional.
# Utilizamos a classe BitsAndBytesConfig para configurar a quantização em 4 bits, o que reduz o uso de memória e acelera o treinamento.
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype="bfloat16",
        bnb_4bit_use_double_quant=True,
)

# Usamos a classe AutoModelForCausalLM para carregar um modelo pré-treinado adequado para modelagem de linguagem causal.
# Parâmetros importantes incluem:
#  - torch_dtype=compute_dtype: Define o tipo de dado para o modelo.
#  - quantization_config=bnb_config: Aplica a configuração de quantização.
#  - device_map="auto": Distribui automaticamente o modelo nos dispositivos disponíveis.
#  - attn_implementation=attn_implementation: Define a implementação da atenção.
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=compute_dtype,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation,
)

# adapta o modelo para o treinamento em k-bits, otimizando ainda mais o desempenho.
model = prepare_model_for_kbit_training(model)

tokenizer = set_tokenizer(model_id)

In [11]:
class LanguageModel:

    def __init__(self, tokenizer, model, device):
        self.tokenizer = tokenizer
        self.model = model
        self.device = device
        if self.tokenizer.pad_token_id is None:
            self.tokenizer.pad_token_id = self.tokenizer.eos_token_id

    def tokenize(self, messages):
        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        model_inputs = tokenizer([text], return_tensors="pt").to(self.device)
        return model_inputs

    def generate(self, messages):
        model_inputs = self.tokenize(messages)
        model_inputs['attention_mask'] = model_inputs['attention_mask'].to(model_inputs['input_ids'].device)
        generated_ids = model.generate(
            model_inputs.input_ids,
            max_new_tokens=512,
            do_sample=True,
            attention_mask=model_inputs['attention_mask'],
            pad_token_id=self.tokenizer.pad_token_id
        )
        generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]
        return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [12]:
%%time

llm = LanguageModel(tokenizer, model, device="cuda")

prompt = "Qual a capital do Brasil?"

messages = [
    # {"role": "user", "content": "Olá. Você é um expert em geografia e vai me ajudar a responder algumas questões."},
    # {"role": "assistent", "content": "Tudo bem! Como posso ajudar?"},
    {"role": "user", "content": prompt},
]

llm.generate(messages)

CPU times: user 21.1 s, sys: 12.2 ms, total: 21.1 s
Wall time: 21.1 s


'No nosso modelo, a capital de um país é a soma das emissões da quantidade total de pessoas na população do país. Aqui eu coloquei os dados:\n1. Quantidade de pessoas vivendo na capital a cada dia. No Brasil, aproximadamente 217 milhões de pessoas vivem e no período de um dia em que o programa comércio e consumo ocorre, o número de pessoas aumenta de 257 milhões. No ano passado, a população brasileira registrou um crescimento maior do que o mundial em relação a um ano.\n2. Custo de trabalho de economia comum. No Brasil, há menos de 1,5 milhões de empregados. O mercado de trabalho está bem concentrado nas segredas, como carros, ôvos e revoluções de geração com capacidades de construção.\n3. Indicadores de crescimento económico. No ano passado, a economia brasileira registrou uma taxa de crescimento anual de 7,2 %. Esta taxa foi também superada pela economia europeia, e a economia mexicana, que aumentaram 8,8% e 4,6% respectivamente.\n4. Dados econômico-gestacional. No ano passado, o val

In [13]:
def format_dataset_chatml(row):
    messages = [
        {
            "content": "Você é um assistente especializado em interpretação de texto",
            "role": "system"
        },
        {
            "content": f"Gere uma pergunta para o seguinte contexto:\n```\n{row['context']}\n```\nPergunta:",
            "role": "user"
        },
        {
            "content": f"{row['question']}",
            "role": "assistant"
        }
    ]

    return {"text": tokenizer.apply_chat_template(messages, add_generation_prompt=False, tokenize=False)}

train_dataset_chatml = train_dataset.map(format_dataset_chatml)
test_dataset_chatml = test_dataset.map(format_dataset_chatml)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [14]:
%%time

sft_config = SFTConfig(
    seed=42,
    output_dir=LOCAL_MODELPATH,
    dataset_text_field="text",
    max_seq_length=512,

    # Training Hyperparameters
    learning_rate=1e-4,
    num_train_epochs=1,
    warmup_ratio=0.1,
    lr_scheduler_type="linear",
    
    # Validation 
    do_eval=True,
    eval_strategy="steps",
    eval_steps=20,

    # Chackpoints
    save_strategy="steps",  # Salvando a cada 100 passos
    save_steps=10,         # Salvando a cada 100 passos
    # save_strategy="epoch",  # Salvando ao final de cada época

    # Loggings
    log_level="warning",
    logging_steps=20,

    # Weights yype
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),

    # Report to Weights And Bias? TensorBoard?
    report_to="none",
)

peft_config = LoraConfig(
        r=16,
        lora_alpha=16,
        lora_dropout=0.05,
        task_type=TaskType.CAUSAL_LM,
        target_modules=['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"],
)

CPU times: user 1.77 ms, sys: 0 ns, total: 1.77 ms
Wall time: 12.5 ms


In [15]:
%%time
trainer = SFTTrainer(
    model,
    train_dataset=train_dataset_chatml,
    eval_dataset=test_dataset_chatml,
    args=sft_config,
    peft_config=peft_config,
    tokenizer=tokenizer,
    callbacks=[SaveCheckpointCallback(), EarlyStoppingCallback( early_stopping_threshold=0.0005), SaveMetricsCallback(LOCAL_MODELPATH)],
)

trainer.train()

trainer.save_model()

Output directory initialized at models/question-generator/qwen-qwen2-0_5b


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.81 GiB. GPU 

In [ ]:
dataset = load_dataset("sentence-transformers/all-nli", "pair-score", split="train")

In [1]:
import torch
import bitsandbytes
import peft
import accelerate
import datasets
import trl


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("torch version:", torch.__version__)
print("bitsandbytes version:", bitsandbytes.__version__)
print("peft version:", peft.__version__)
print("accelerate version:", accelerate.__version__)
print("datasets version:", datasets.__version__)
print("trl version:", trl.__version__)
print(f"Device name: '{torch.cuda.get_device_name()}'")
print("Device:", device)
print(f"Device properties: '{torch.cuda.get_device_properties(torch.cuda.current_device())}'")
print("Suporta bfloat16." if torch.cuda.is_bf16_supported() else "Não suporta bfloat16.")

Device name: 'NVIDIA GeForce RTX 2060 SUPER'
Device properties: '_CudaDeviceProperties(name='NVIDIA GeForce RTX 2060 SUPER', major=7, minor=5, total_memory=7957MB, multi_processor_count=34)'
Suporta bfloat16.


In [2]:
warnings.filterwarnings('ignore')
login(token=os.environ["HUGGINGFACE_TOKEN"])

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
class LanguageModel:

    def __init__(self, tokenizer, model, device):
        self.tokenizer = tokenizer
        self.model = model
        self.device = device
        if self.tokenizer.pad_token_id is None:
            self.tokenizer.pad_token_id = self.tokenizer.eos_token_id

    def tokenize(self, messages):
        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        model_inputs = tokenizer([text], return_tensors="pt").to(self.device)
        return model_inputs

    def generate(self, messages):
        model_inputs = self.tokenize(messages)
        model_inputs['attention_mask'] = model_inputs['attention_mask'].to(model_inputs['input_ids'].device)
        generated_ids = model.generate(
            model_inputs.input_ids,
            max_new_tokens=4000,
            do_sample=True,
            attention_mask=model_inputs['attention_mask'],
            pad_token_id=self.tokenizer.pad_token_id
        )
        generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]
        return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [4]:
%%time
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    # bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", quantization_config=quantization_config)

tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

CPU times: user 22.3 s, sys: 5.31 s, total: 27.6 s
Wall time: 7.46 s


In [5]:
%%time
llm = LanguageModel(tokenizer, model, device="cuda")

prompt = "Quem foi Einstein?"

messages = [
    {"role": "user", "content": "Olá"},
    {"role": "assistent", "content": "Tudo bem? Como posso ajudar?"},
    {"role": "user", "content": prompt},
]

llm.generate(messages)

CPU times: user 17.9 s, sys: 162 ms, total: 18 s
Wall time: 18 s


'Albert Einstein foi um físico teórico alemão-estadunidense nascido na Alemanha em 1879 e falecido em 1955. Ele é considerado um dos maiores cientistas do século XX e é amplamente reconhecido como o pai da teoria da relatividade.\n\nEinstein é conhecido por suas contribuições fundamentais para a física, incluindo a teoria da relatividade restrita (1905) e a teoria da relatividade geral (1915). Sua teoria da relatividade mudou a forma como a física e a astronomia eram entendidas e abriu caminho para novas áreas de pesquisa.\n\nAlguns de seus principais contributos incluem:\n\n1. **Teoria da relatividade restrita** (1905): Descreveu a velocidade da luz como uma constante universal e introduziu o conceito de tempo e espaço como relativos.\n2. **Equação E=mc²** (1905): Mostrou que a energia (E) é igual à massa (m) multiplicada pela velocidade da luz ao quadrado (c²).\n3. **Teoria da relatividade geral** (1915): Descreveu a gravidade como uma curvatura do espaço-tempo causada pela massa e e

In [8]:

# Texto de entrada
input_text = "Quanto é 1 + 1?"
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Gerar texto
output = model.generate(input_ids, max_length=50, num_return_sequences=1)

# Decodificar e imprimir o resultado
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Quanto é 1 + 1? 2
Quanto é 1 + 2? 3
Quanto é 2 + 1? 3
Quanto é 2 + 2? 4
Qu
